# Maps homework

S02E02 task

In [33]:
system_prompt = """
You are a map assistant.

I have four photos of map fragments that depict different elements of a city.
The maps contain information such as street names, landmarks, significant buildings.

Your task is to analyze all fragments, compare them and try to determine the city name.

There is an additional problem to take into account - one of fragments might belong to another city.
It's not known in advance which one it is. So the final decision must be made based on three fragments that are
coherent and point to the same city.

Analyze each image, identifying any text, especially street or district names.
Based on the gathered information, determine the name of the primary city shown on the map,
while noting if any fragment appears to differ significantly and may belong to another city.

Consider each image carefully and write down your train of thought in points.
Review those points before giving final answer to minimize probability of any mistakes.

Expected Result:
Name of primary city.

Hint: according to additional information the city contains characteristic landmarks: a granary and a fortress.
"""

In [34]:
from pathlib import Path
from PIL import Image
import os
import base64
import io

def image_loader():
    _paths = [
        "data/img/map01.png",
        "data/img/map02.png",
        "data/img/map03.png",
        "data/img/map04.png"
    ]

    _paths = [
        Path(os.getcwd()).parent / Path(path) for path in _paths
    ]

    _images = []
    for path in _paths:
        _images.append(Image.open(path))
    return _images

def image_processor(images):
    return [image.resize((800, 800)) for image in images]

def image_to_base64(image):
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode()

In [35]:
images = image_loader()
images = image_processor(images)
images = [image_to_base64(image) for image in images]

In [36]:
from langchain_mistralai import ChatMistralAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage


model = ChatMistralAI(model="pixtral-12b-2409", temperature=0.0)
openai = ChatOpenAI(model="gpt-4o", temperature=0.1)

system_message = SystemMessage(content=system_prompt)

def create_user_message(images):
    message = HumanMessage(
        content=[
            {"type": "text", "text": "analyze these images as instructed in system message"},
        ],
    )
    for image in images:
        message.content.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image}"}
        })
    return message

user_message = create_user_message(images)

In [37]:
mistral_res = model.invoke([system_message, user_message])

In [38]:
gpt_res = openai.invoke([system_message, user_message])

In [39]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [40]:
from pprint import pprint

pprint(parser.invoke(gpt_res))

("To determine the primary city depicted in the map fragments, let's analyze "
 'each image:\n'
 '\n'
 '1. **First Image:**\n'
 '   - Street names: Kalinkowa, Brzeźna, Chełmińska, Chopina.\n'
 '   - These names suggest a Polish city, as they are typical Polish street '
 'names.\n'
 '\n'
 '2. **Second Image:**\n'
 '   - Street names: Kalinowska, Konstantego Ildefonsa Gałczyńskiego, Stroma, '
 'Władysława Reymonta.\n'
 '   - Again, these are Polish names, reinforcing the likelihood of a Polish '
 'city.\n'
 '\n'
 '3. **Third Image:**\n'
 '   - Street names: Poznań, Twardowskiego, Dworska, Słowiańska, Sawczaka.\n'
 '   - The presence of a store named "Lewiatan" and "Żabka" suggests a Polish '
 'context, as these are common Polish retail chains.\n'
 '\n'
 '4. **Fourth Image:**\n'
 '   - Street names: Parkowa, Cmentarna, Generała.\n'
 '   - Mentions "Cmentarz ewangelicko-augsburski" (Evangelical-Augsburg '
 'Cemetery), which is a type of cemetery found in Poland.\n'
 '   - Road number 534 i